In [1]:
from torch.utils.data import DataLoader, RandomSampler
from torchvision import models
import torch
import timm
from tiny_vit import tiny_vit_5m_224, tiny_vit_11m_224, tiny_vit_21m_224
import pandas as pd

/home/ajk4yq/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ajk4yq/Documents/MSDS/Semester5/DS5110/Big-Data-Systems/Team 11/tiny_vit.py:641: UserWarning: Overwriting tiny_vit_5m_224 in registry with tiny_vit.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def tiny_vit_5m_224(pretrained=False, **kwargs):
/home/ajk4yq/Documents/MSDS/Semester5/DS5110/Big-Data-Systems/Team 11/tiny_vit.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with tiny_vit.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def tiny_vit_11m_224(pretrained=False, **kwargs):
/home/ajk4yq/Documents/MSDS/Semester5/DS5110/Big-Data-Systems/Team 11/tin

In [2]:
# load mobilenet_v3_large
mobilenet_v3_large = models.mobilenet_v3_large(weights="IMAGENET1K_V2")

# load mobilenet_v3_small
mobilenet_v3_small = models.mobilenet_v3_small(weights="IMAGENET1K_V1")

# load mnasnet0_5
mnasnet0_5 = models.mnasnet0_5(weights="IMAGENET1K_V1")

# load shufflenet_v2_x0_5
shufflenet_v2_x0_5 = models.shufflenet_v2_x0_5(weights="IMAGENET1K_V1")

# load shufflenet_v2_x1_0
shufflenet_v2_x1_0 = models.shufflenet_v2_x1_0(weights="IMAGENET1K_V1")

# load microsoft's tiny_vit
num_classes = 120

tiny_vit_5m = tiny_vit_5m_224(pretrained=True)
tiny_vit_5m.head = torch.nn.Linear(tiny_vit_5m.head.in_features, num_classes)

In [3]:
def count_parameters(model):
    num_params = 0
    size_model = 0
    for param in model.parameters():
        num_params +=  param.numel()
        if param.data.is_floating_point():
            size_model += param.numel() * torch.finfo(param.data.dtype).bits
        else:
            size_model += param.numel() * torch.iinfo(param.data.dtype).bits
    size_model = f'{size_model/8e6:.2f} MB'
    return (num_params, size_model)



In [10]:
model_list = [mnasnet0_5, mobilenet_v3_large, mobilenet_v3_small, shufflenet_v2_x0_5, shufflenet_v2_x1_0, tiny_vit_5m]
model_names = ['mnasnet0_5', 'mobilenet_v3_large', 'mobilenet_v3_small','shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'tiny_vit_5m']
model_zip = zip(model_list, model_names)

model_desc_df = pd.DataFrame(columns=['Model', 'Num_Params', 'Size_MB'])

for model_tup in model_zip:
    params, size = count_parameters(model_tup[0])
    model_desc_df.loc[len(model_desc_df.index)] = [model_tup[1], params, size]

In [11]:
model_desc_df.to_csv('model_descript.csv')

In [9]:
#create markdown table
print('| Model | # of Parameters | Size (MB)|')
print('| ----- | --------------- | -------- |')

model_list = [mnasnet0_5, mobilenet_v3_large, mobilenet_v3_small, shufflenet_v2_x0_5, shufflenet_v2_x1_0, tiny_vit_5m]
model_names = ['mnasnet0_5', 'mobilenet_v3_large', 'mobilenet_v3_small','shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'tiny_vit_5m']
model_zip = zip(model_list, model_names)

for model_tup in model_zip:
    params, size = count_parameters(model_tup[0])
    print(f'| {model_tup[1]} | {params} | {size} |')

| Model | # of Parameters | Size (MB)|
| ----- | --------------- | -------- |
| mnasnet0_5 | 2218512 | 8.87 MB |
| mobilenet_v3_large | 5483032 | 21.93 MB |
| mobilenet_v3_small | 2542856 | 10.17 MB |
| shufflenet_v2_x0_5 | 1366792 | 5.47 MB |
| shufflenet_v2_x1_0 | 2278604 | 9.11 MB |
| tiny_vit_5m | 5110284 | 20.44 MB |
